In [ ]:
!pip install pandas

In [60]:
import pickle 
titles = pickle.load(open("benchmark_graph_titles.pkl", "rb"))
embeddings = pickle.load(open("benchmark_graph_embeddings.pkl", "rb"))

In [61]:
import numpy as np
embeddings.shape

(2381184, 256)

In [62]:
titles = np.array(titles)

In [63]:
# check titles in the embeddings
import pandas as pd
csv = pd.read_csv('benchmark_data.csv')

In [64]:
csv

,search_doi,search_title,result_doi,result_title
0,arXiv:2003.01332,Heterogeneous Graph Transformer,arXiv:1703.06103,Modeling Relational Data with Graph Convolutio...
1,arXiv:2003.01332,Heterogeneous Graph Transformer,arXiv:1710.10903,Graph Attention Networks
2,arXiv:2003.01332,Heterogeneous Graph Transformer,arXiv:1904.05811,Relational Graph Attention Networks
3,arXiv:1902.10197,RotatE: Knowledge Graph Embedding by Relationa...,arXiv:1412.6575,Embedding Entities and Relations for Learning ...
4,arXiv:1902.10197,RotatE: Knowledge Graph Embedding by Relationa...,arXiv:1606.06357,Complex Embeddings for Simple Link Prediction
5,arXiv:1902.10197,RotatE: Knowledge Graph Embedding by Relationa...,arXiv:2107.07842,A Survey of Knowledge Graph Embedding and Thei...
6,arXiv:2306.11264,GraphGLOW: Universal and Generalizable Structu...,arXiv:2310.05174,GSLB: The Graph Structure Learning Benchmark
7,arXiv:2306.11264,GraphGLOW: Universal and Generalizable Structu...,arXiv:2105.09111,Self-supervised Heterogeneous Graph Neural Net...
8,arXiv:2306.11264,GraphGLOW: Universal and Generalizable Structu...,arXiv:1801.07829,Dynamic Graph CNN for Learning on Point Clouds
9,arXiv:2011.08843,Design Space for Graph Neural Networks,arXiv:2203.00112,GraphWorld: Fake Graphs Bring Real Insights fo...


In [65]:
# check all titles in csv
titles = [title.lower().replace('\n','').replace('  ',' ') for title in titles]
for title in csv['search_title']:
    if title.lower() not in titles:
        print(title)
        
for title in csv['result_title']:
    if title.lower() not in titles:
        print(title)

In [66]:
titles = pd.Series(titles)

In [67]:
# get index of title
def get_index(title):
    return titles.loc[titles == title].index[0]

get_index(csv['search_title'][0].lower())

2371351

In [68]:
# get cosine similarity between target and all titles
# use numpy
from numpy.linalg import norm
def get_cosine_similarity(target, embeddings):
    target = target.reshape(1,-1)
    target = target / norm(target)
    embeddings = embeddings / norm(embeddings, axis=1).reshape(-1,1)
    result =  np.dot(target, embeddings.T) # dot product is unnormalized cosine
    return result[0]

cosine_sims = {}
for seach_title in csv['search_title'].unique():
    print(seach_title)
    cosine_sims[seach_title] = get_cosine_similarity(embeddings[get_index(seach_title.lower())], embeddings)

Heterogeneous Graph Transformer
RotatE: Knowledge Graph Embedding by Relational Rotation in Complex Space
GraphGLOW: Universal and Generalizable Structure Learning for Graph Neural Networks
Design Space for Graph Neural Networks
Semi-Supervised Classification with Graph Convolutional Networks
Deeper Insights into Graph Convolutional Networks for Semi-Supervised Learning
How Powerful are Graph Neural Networks?
Detection of Anomalies in Large Scale Accounting Data using Deep Autoencoder Networks
Analyzing Business Process Anomalies Using Autoencoders
Understanding Graph Convolutional Networks for Text Classification


In [69]:
cosine_sims['Heterogeneous Graph Transformer'][get_index('Heterogeneous Graph Transformer'.lower())]

1.0

In [70]:
indices = np.argsort(cosine_sims['Heterogeneous Graph Transformer'])[::-1]

In [71]:
total_average_rank = 0
for seach_title in csv['search_title'].unique():
    result_titles = csv.loc[csv['search_title'] == seach_title]['result_title']
    number_result_titles = len(result_titles)
    indices = np.argsort(cosine_sims[seach_title])[::-1]
    print('------------')
    print('Search title: ', seach_title)
    avg_rank = 0
    for i in range(number_result_titles):
        result_title = result_titles.iloc[i]
        result_title_index = get_index(result_title.lower())
        rank = np.argwhere(indices==result_title_index)[0][0] + 1
        
        print('rank', rank,'Result title',result_title)
        print('----------------------')
        avg_rank += rank
    print('Average rank: ', round(avg_rank/number_result_titles,2),'out of', len(titles))
    total_average_rank += avg_rank/number_result_titles

print('Total average rank: ', round(total_average_rank/len(csv['search_title'].unique()),2),'out of', len(titles))

------------
Search title:  Heterogeneous Graph Transformer
rank 84701 Result title Modeling Relational Data with Graph Convolutional Networks
----------------------
rank 766 Result title Graph Attention Networks
----------------------
rank 26487 Result title Relational Graph Attention Networks
----------------------
Average rank:  37318.0 out of 2381184
------------
Search title:  RotatE: Knowledge Graph Embedding by Relational Rotation in Complex Space
rank 114391 Result title Embedding Entities and Relations for Learning and Inference in Knowledge Bases
----------------------
rank 97676 Result title Complex Embeddings for Simple Link Prediction
----------------------
rank 119252 Result title A Survey of Knowledge Graph Embedding and Their Applications
----------------------
Average rank:  110439.67 out of 2381184
------------
Search title:  GraphGLOW: Universal and Generalizable Structure Learning for Graph Neural Networks
rank 31980 Result title GSLB: The Graph Structure Learning B

## Result:
Average rank across all queries: 81752.15  out of 2381184